# building mini chat-gpt

Video: [Let's build GPT: from scratch, in code, spelled out.](https://youtu.be/kCc8FmEb1nY?list=PLAqhIrjkxbuWI23v9cThsA9GvCAUhRvKZ&t=561) - Andrej Karpathy

Paper: [Attention is All You Need](https://arxiv.org/abs/1706.03762)

<img src="gpt.png" width="600" height="800" style="margin-left:auto; margin-right:auto"/>

## Part 1: explore the dataset

In [1]:
# imports
import torch

In [2]:
with open('kanye_lyrics.txt', 'r') as f:
    lyrics = f.read()

In [3]:
print('Length of dataset:    ', len(lyrics), 'characters')

Length of dataset:     353441 characters


### 1.1: view the data

In [4]:
print('First 1000 characters:\n ', lyrics[:1000])

First 1000 characters:
  ﻿[Chorus]
Sing every hour (Every hour, 'til the power)
Every minute (Every minute, of the Lord)
Every second (Every second, comes)
Sing each and every millisecond (Down)
We need you (We need you, sing 'til the power)
We need you (We need you, of the Lord)
We need you (Comes)
Oh, we need you (Down)


[Verse]
Sing 'til the power of the Lord comes down
Sing 'til the power of the Lord comes down
Sing 'til the power of the Lord comes down (Let everything that have breath praise God)
Sing 'til the power of the Lord comes down ('Cause when we sing the glory of the Lord comes down, down)
Sing 'til the power of the Lord comes down (Praising the Lord, praise God in the sanctuary)
Sing 'til the power of the Lord comes down (For His mighty works and excellent grace and His mighty power, yeah)
Sing 'til the power of the Lord comes down (Sing 'til the power of the Lord falls down)
Sing 'til the power of the Lord comes down (We are to sing 'til the power of the Lord comes dow

### 1.2 vocabulary size

In [34]:
chars = sorted(list(set(lyrics)))
vocab_size = len(chars)
print(''.join(c for c in chars))
print('Number of unique characters: ', vocab_size)


 !"#$&'()*+,-./0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]abcdefghijklmnopqrstuvwxyzÁáéñóöúāŐ–—‘’“”…⁠﻿
Number of unique characters:  101


'\n'

## Part 2: encoding and decoding

### 2.1 encode and decode functions

In [6]:
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s : [stoi[c] for c in s]
decode = lambda l : ''.join([itos[c] for c in l])
print(encode('hii there'))
print(decode(encode('hii there')))

[64, 65, 65, 1, 76, 64, 61, 74, 61]
hii there


In [7]:
print(encode("Ándale, ándale E.I, E.I, uh-oh"))
print(decode(encode("Ándale, ándale E.I, E.I, uh-oh")))

[83, 70, 60, 57, 68, 61, 12, 1, 84, 70, 60, 57, 68, 61, 1, 33, 14, 37, 12, 1, 33, 14, 37, 12, 1, 77, 64, 13, 71, 64]
Ándale, ándale E.I, E.I, uh-oh


### 2.2 encode the dataset

In [8]:
data = torch.tensor(encode(lyrics), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([353441]) torch.int64
tensor([100,  55,  31,  64,  71,  74,  77,  75,  56,   0,  47,  65,  70,  63,
          1,  61,  78,  61,  74,  81,   1,  64,  71,  77,  74,   1,   8,  33,
         78,  61,  74,  81,   1,  64,  71,  77,  74,  12,   1,   7,  76,  65,
         68,   1,  76,  64,  61,   1,  72,  71,  79,  61,  74,   9,   0,  33,
         78,  61,  74,  81,   1,  69,  65,  70,  77,  76,  61,   1,   8,  33,
         78,  61,  74,  81,   1,  69,  65,  70,  77,  76,  61,  12,   1,  71,
         62,   1,  76,  64,  61,   1,  40,  71,  74,  60,   9,   0,  33,  78,
         61,  74,  81,   1,  75,  61,  59,  71,  70,  60,   1,   8,  33,  78,
         61,  74,  81,   1,  75,  61,  59,  71,  70,  60,  12,   1,  59,  71,
         69,  61,  75,   9,   0,  47,  65,  70,  63,   1,  61,  57,  59,  64,
          1,  57,  70,  60,   1,  61,  78,  61,  74,  81,   1,  69,  65,  68,
         68,  65,  75,  61,  59,  71,  70,  60,   1,   8,  32,  71,  79,  70,
          9,   0,  51,  61,   1

### 2.3: data split

In [9]:
train_split = 0.9
n = int(train_split * len(data))
train_data = data[:n]
val_data = data[n:]

### 2.4 note on training

When we train the model, we will not process the entire dataset at once, nor will we train on single character pairs. Instead, we will decide on a size of a block of text to look at.

In [10]:
# start with block_size = 8
block_size = 8
train_data[:block_size+1]

tensor([100,  55,  31,  64,  71,  74,  77,  75,  56])

In [11]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"When input is {context} the target is {target}")
for t in range(block_size):
    context = decode(x[:t+1].tolist())
    target = decode([y[t].item()])
    print(f"When input is {context} the target is {target}")

When input is tensor([100]) the target is 55
When input is tensor([100,  55]) the target is 31
When input is tensor([100,  55,  31]) the target is 64
When input is tensor([100,  55,  31,  64]) the target is 71
When input is tensor([100,  55,  31,  64,  71]) the target is 74
When input is tensor([100,  55,  31,  64,  71,  74]) the target is 77
When input is tensor([100,  55,  31,  64,  71,  74,  77]) the target is 75
When input is tensor([100,  55,  31,  64,  71,  74,  77,  75]) the target is 56
When input is ﻿ the target is [
When input is ﻿[ the target is C
When input is ﻿[C the target is h
When input is ﻿[Ch the target is o
When input is ﻿[Cho the target is r
When input is ﻿[Chor the target is u
When input is ﻿[Choru the target is s
When input is ﻿[Chorus the target is ]


### 2.5 data loader

In [12]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    # generate  a small batch of data of inputs x and target y
    data = train_data if split=='train' else val_data
    ix = torch.randint(len(data)-block_size, size=(batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"When input is {context.tolist()} the target is {target}")


inputs:
torch.Size([4, 8])
tensor([[61,  0, 47, 64, 61,  1, 79, 57],
        [ 1, 60, 71, 70,  7, 76,  1, 79],
        [71, 70,  1, 69, 81,  1, 79, 57],
        [76, 81,  0, 51, 61,  1, 61, 74]])
targets:
torch.Size([4, 8])
tensor([[ 0, 47, 64, 61,  1, 79, 57, 70],
        [60, 71, 70,  7, 76,  1, 79, 71],
        [70,  1, 69, 81,  1, 79, 57, 65],
        [81,  0, 51, 61,  1, 61, 74, 57]])
----
When input is [61] the target is 0
When input is [61, 0] the target is 47
When input is [61, 0, 47] the target is 64
When input is [61, 0, 47, 64] the target is 61
When input is [61, 0, 47, 64, 61] the target is 1
When input is [61, 0, 47, 64, 61, 1] the target is 79
When input is [61, 0, 47, 64, 61, 1, 79] the target is 57
When input is [61, 0, 47, 64, 61, 1, 79, 57] the target is 70
When input is [1] the target is 60
When input is [1, 60] the target is 71
When input is [1, 60, 71] the target is 70
When input is [1, 60, 71, 70] the target is 7
When input is [1, 60, 71, 70, 7] the target is 76
W

## Part 3: Implementing networks

### 3.1 the bigram language model

In [33]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLangaugeModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.vocab_size = vocab_size
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # idx and targets are both (B,T) tensors of integers
        logits = self.token_embedding_table(idx) # (B,T,C)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            # idx is a (B,T) tensor of integers
            logits, loss = self(idx) # (B,T,C)
            # take only last character of each example
            logits = logits[:, -1, :] # Last characters (B,C)
            # generate probabilities along C dimension
            probs = F.softmax(logits, dim=-1) 
            # sample from the distribution
            next_idx = torch.multinomial(probs, num_samples=1) # (B,1)
            # append
            idx = torch.cat((idx, next_idx), dim=1)
        return idx

    
    
    
    
m = BigramLangaugeModel(vocab_size)
logits, loss = m(xb, yb)
# print(logits.shape)
# print(loss)
print(m.generate(xb, 10)) # works!

tensor([[61,  0, 47, 64, 61,  1, 79, 57, 63, 20, 96, 68, 92, 49, 83, 50, 91, 85],
        [ 1, 60, 71, 70,  7, 76,  1, 79,  0, 99, 43, 66, 59, 56, 99, 39,  8, 89],
        [71, 70,  1, 69, 81,  1, 79, 57, 43, 66, 26,  1, 43, 82, 19, 79, 35, 68],
        [76, 81,  0, 51, 61,  1, 61, 74, 49, 69, 20,  9, 94, 54, 12, 49, 54, 54]])


In [40]:
# test the model before training:
print('Before training:')
print('----')
print(decode(m.generate(torch.zeros((1,1), dtype=torch.long), 50)[0].tolist()))

Before training:
----

Ct875ö&c;bLCZ#⁠$O]2é–4Q.
eY–9JS[$(+⁠hQ:kxCs047gŐ?b


<strong>question: what do B, T, C represent about relating to the tensor dimensions?</strong>

Guess: B is the batch size, T is the 'block size' (in our case, 4), and C is the number of 'channels', which in our 
case of the logits, corresponds to the dimensionality of the embedding table.

### 3.2 training the bigram model

In [41]:
# add an optimizer
opt = torch.optim.AdamW(m.parameters(), lr=1e-3)

#### optimization loop


In [62]:
batch_size = 32
max_steps = 100000
for step in range(max_steps):

    # sample
    xb, yb = get_batch('train')

    # forward
    logits, loss = m(xb, yb)
    opt.zero_grad(set_to_none=True)
    loss.backward()
    opt.step()

    print(loss.item())

2.4934463500976562
2.3979737758636475
2.4289777278900146
2.4395792484283447
2.475491523742676
2.4161343574523926
2.449674367904663
2.5793912410736084
2.528151750564575
2.463026762008667
2.434858560562134
2.4419400691986084
2.5193936824798584
2.389005661010742
2.5645463466644287
2.477537155151367
2.339337110519409
2.3810009956359863
2.504850387573242
2.4704396724700928
2.5377676486968994
2.551757574081421
2.443244218826294
2.4084346294403076
2.474635362625122
2.485530138015747
2.4125537872314453
2.5460901260375977
2.463956117630005
2.560131311416626
2.4161570072174072
2.5515823364257812
2.572497844696045
2.385486602783203
2.443502902984619
2.4278297424316406
2.527238607406616
2.454572916030884
2.473768472671509
2.593329906463623
2.4024360179901123
2.5564393997192383
2.3819167613983154
2.4575793743133545
2.378713607788086
2.4391751289367676
2.515401601791382
2.3773305416107178
2.5207693576812744
2.58634352684021
2.5637965202331543
2.428037643432617
2.4321675300598145
2.5701286792755127
2

In [65]:
# test the model after training:
print('after training:')
print('----')
print(decode(m.generate(torch.zeros((1,1), dtype=torch.long), 1000)[0].tolist()))

after training:
----

I gng te Plye
[Chasc m g f]
An p? Weet beredo izesomatomolan, Prel t t tor n'thive Iny now ms y ss, bor "

I s ge nom astraresangil my, clifowhouplwe tecacthem d?
Th resor
Manga at
Bor thee bl ju ars ce n te bangrya moxan', ms shiarir he, ggse juede, I' ghey dougohe t s In pokinga waparigheairacod seme ootoupucaninickedou "Kase jos


Frsouri]
Ou't an wane ait t thas
Sar ind
Gowiveym w boust h n ustiangougerod
[I (Mepidese
Ő; boke n
B I'sele
Hit've e s: higslo It bout thon
We, d courexa oin defee e g it'tidin]
Jobe seve fryssthanak cury s Whhu gs, wa tezyo ally lotazilowesh cohithernikeelon ctherky, a & Kaingeandefovero atho: m ut besuthitolbl fos mpin'mere s

E.' knd our gghis Ro p ith s aikölsmory
[Hanceten ourythe y, awasth)
Can fou cke t
Noow
Leice, you thithy p sslps, ou y
[Camoustht ckerlou d gg fean yoknifffr pu ck waksh, I' Lind d diomemyin
Thit I sa
3 st, she t an myous t' Hamout cive, tutis ss thon Jou ithin, t mmabut, avifyoutthe Ithem me go mamanin'ther